### WEATHER API
  Instructions:
OpenWeather API
Create an account here at: https://openweathermap.org/ . You will use their API to get weather forecast data
After you have created the free account, request an API key. The key enables you to connect to their API and pull weather data.

  Weather Function
Create a function which will make requests to the OpenWeather Current Weather Data API to get weather information using the latitude and longitude coordinates for a city.
Parse the response from the API request and extract the following information about a city: id,name,country,main description,temperature,minimum temperature,maximum temperature,feels_like,humidity and wind speed.

The function should return a dictionary of the format: {
"id": 802,
"city": "Skopje",
"latitude": 42,
"longtitude": 21.4333,
"country": 25,
"description": "scattered clouds",
"temp":290.97,
"temp_min": 293.97,
"temp_max": 293.97,
"feels_like": 293.23,
"humidity": 43,
"wind_speed": 2.06
}
Design the function in a way that it should work for any given pair of values for latittude and longitute coordinates.
Handle failures in weather function

A failure in this case as an example would be considered passing wrong values for latitude and longtitude. For this reason, modify the weather function you developed before to gracefully fail if wrong values for latitude and longtitude are passed.

The function should capture the response error message from the API call. In these cases, record this information as a new entry into the dataframe . Explore the structure of this dataframe to see what information you need to capture and what columns you need to populate. If required, process the response further to populate the columns accordingly.

In [98]:
import requests
import pandas as pd

successRequests = []
failedRequests = []

APIkey = '3b44ca87fb86045dc73d251817916e47'
lat = '42.0062'
lon = '20.9698'
#r = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={APIkey}')  #forecast api
r = requests.get(f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={APIkey}')    #weather api


#print(r.country)
print(r.url)
print(r.status_code)
#print(r.text)
data = r.json()


def getWeather(lat, lon, APIkey):

    try:
        r = requests.get(f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={APIkey}')
        r.raise_for_status() #jump to except code
        weatherData = r.json()

        weather = {
            "id": weatherData['id'],
            "city": weatherData['name'],
            "latitude": weatherData['coord']['lat'],
            "longtitude": weatherData['coord']['lon'], # Spelling from your prompt
            "country": weatherData['sys']['country'],
            "description": weatherData['weather'][0]['description'], # 'weather' is a list!
            "temp": weatherData['main']['temp'],
            "temp_min": weatherData['main']['temp_min'],
            "temp_max": weatherData['main']['temp_max'],
            "feels_like": weatherData['main']['feels_like'],
            "humidity": weatherData['main']['humidity'],
            "wind_speed": weatherData['wind']['speed']
        }

        return(weather)

    except requests.exceptions.HTTPError as e:
        errorData = e.response.json()

        error = {
            'Error Code': errorData['cod'],
            'Error Message': errorData['message'],
            'latitute': lat,
            'longitude': lon
        }
        return (errorData, error)

getWeather(42,  21.4333, APIkey)

latlonDict = [
    (42, 21.4333),
    (42,24),
    (41,41),
    (999, 53),
    (999,999),
    (41, -500)
]

for lat, lon in latlonDict:
    result = getWeather(lat, lon, APIkey)

    if isinstance(result, tuple):
        errorData = result[1]
        failedRequests.append(errorData)

    else:
        successRequests.append(result)


df = pd.DataFrame(successRequests)
dfErrors = pd.DataFrame(failedRequests)

#getWeather(999, 42, APIkey)

#getWeather(999, 9999, APIkey)

print(df)
print(dfErrors)

df.to_csv(f"../output/apiRequest.csv",index=False)
dfErrors.to_csv(f"../output/errorRequests.csv",index=False)




https://api.openweathermap.org/data/2.5/weather?lat=42.0062&lon=20.9698&appid=3b44ca87fb86045dc73d251817916e47
200
       id         city  latitude  longtitude country      description    temp  \
0  785842       Skopje        42     21.4333      MK    broken clouds  286.97   
1  729519     Ladzhene        42     24.0000      BG  overcast clouds  281.60   
2  749841  Çamlıhemşin        41     41.0000      TR  overcast clouds  291.54   

   temp_min  temp_max  feels_like  humidity  wind_speed  
0    286.97    286.97      286.29        72        0.51  
1    281.60    281.60      280.97        84        1.56  
2    291.54    291.54      291.37        74        1.31  
  Error Code    Error Message  latitute  longitude
0        400   wrong latitude       999         53
1        400   wrong latitude       999        999
2        400  wrong longitude        41       -500


In [ ]:
id = data['id']
name = data['name']
country = data['sys']['country']
mainDescription = data['weather'][0]['description']
temperature = data['main']['temp']
minTemp = data['main']['temp_min']
maxTemp = data['main']['temp_max']
feelsLike = data['main']['feels_like']
humidity = data['main']['humidity']
windSpeed = data['wind']['speed']

print(id, name, country, mainDescription, temperature, minTemp, maxTemp, feelsLike, humidity, windSpeed)

#print(r.json())

#print(data['list'][0]['weather'][0]['description'])
#print(data['list'][0]['temp'])
#print(data['city'])